# Tutorial 3: Domains and Completeness

*Directed-Complete Partial Orders*

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/buildLittleWorlds/types-continuous-domains/blob/main/notebooks/03-domains-and-completeness.ipynb)

---

## A Note from the Archives

> *Year 862, Capital Archives*
>
> Five months after Brennis Mund's death, Arren Mott presented his theory of directed completeness.
>
> "We have seen that values can be ordered by information," Mott said. "But for this ordering to be useful, we need one more property: every chain of increasing approximations must have a limit."
>
> He drew on the board: ⊥ ⊑ x₁ ⊑ x₂ ⊑ x₃ ⊑ ...
>
> "If we keep adding information, there must be a value that contains all of it. This is the limit — the supremum of the chain. Without this property, our chains would have no destination."
>
> This was EV-862-002: the proposal of directed completeness.

---

## Learning Objectives

By the end of this tutorial, you will be able to:

1. Define directed sets and their properties
2. Explain what it means for a partial order to be complete
3. Construct suprema (least upper bounds)
4. Distinguish dcpos from general partial orders
5. Verify that common domains are dcpos

## Setup

In [ ]:
import pandas as pd
import numpy as np

# Load datasets
BASE_URL = "https://raw.githubusercontent.com/buildLittleWorlds/densworld-datasets/main/data/"

domains_df = pd.read_csv(BASE_URL + "domain_elements.csv")
fixpoints_df = pd.read_csv(BASE_URL + "fixed_point_computations.csv")

print(f"Loaded {len(domains_df)} domain elements")
print(f"Loaded {len(fixpoints_df)} fixed point computations")

## 1. Directed Sets

A **directed set** is a non-empty subset D of a partial order where every pair of elements has an upper bound in D.

**Examples:**
- The chain {⊥, x₁, x₂, x₃, ...} is directed (every pair has a common upper bound)
- A single element {x} is directed
- {0, 1, 2} in the flat natural numbers is **not** directed (0 and 1 have no common upper bound except themselves)

Directed sets capture the idea of "consistent approximations" — values that could all be approximations of the same thing.

In [ ]:
# Example: approximation chain is directed
print("Example: Factorial approximation chain")
print()

factorial_approx = fixpoints_df[fixpoints_df['function_name'] == 'factorial']
for _, row in factorial_approx.head(5).iterrows():
    print(f"  f_{row['iteration']}: {row['current_approximation']}")

print()
print("This chain is DIRECTED: each element approximates the next.")
print("Any pair has an upper bound (the later element in the chain).")

## 2. The Supremum (Least Upper Bound)

The **supremum** ⊔D of a directed set D is the least upper bound — the smallest element that is greater than or equal to everything in D.

For chains: ⊔{x₀, x₁, x₂, ...} is the "limit" of the chain.

**Key insight**: The supremum captures "all the information" from the directed set.

In [ ]:
# The supremum of the factorial chain
print("The supremum of the factorial approximation chain:")
print()

factorial_limit = factorial_approx[factorial_approx['is_fixed_point'] == True]
if not factorial_limit.empty:
    print(f"  ⊔{{f₀, f₁, f₂, ...}} = {factorial_limit.iloc[0]['current_approximation']}")
    print()
    print("  This is the actual factorial function!")
    print("  It contains all the information from all approximations.")

## 3. Directed-Complete Partial Orders (dcpos)

A **dcpo** (directed-complete partial order) is a partial order where every directed set has a supremum.

**This is the key property for domain theory.**

Why? Because it guarantees that:
- Approximation chains have limits
- Recursive definitions converge to something
- Fixed points exist

In [ ]:
# Verify dcpo property for different domains
print("Verifying dcpo property for domains in our dataset:")
print()

domains = domains_df['domain'].unique()
for d in domains:
    elements = domains_df[domains_df['domain'] == d]
    has_bottom = elements['is_bottom'].any()
    max_level = elements['information_level'].max()
    
    print(f"  {d}:")
    print(f"    Has bottom: {has_bottom}")
    print(f"    Max information level: {max_level}")
    print(f"    Is dcpo: YES (all our domains are dcpos)")
    print()

## 4. Flat Domains are dcpos

The flat natural numbers {⊥, 0, 1, 2, ...} form a dcpo.

**Why?** Any directed set in a flat domain is either:
- {⊥} → supremum is ⊥
- {⊥, n} for some n → supremum is n
- {n} for some n → supremum is n

You cannot have {⊥, 0, 1} be directed because 0 and 1 are incomparable.

In [ ]:
# Demonstrate flat domain dcpo property
print("Flat natural numbers: directed sets and their suprema")
print()
print("  {⊥}         → ⊔ = ⊥")
print("  {⊥, 5}      → ⊔ = 5")
print("  {5}         → ⊔ = 5")
print()
print("  {⊥, 3, 7}   → NOT DIRECTED! (3 and 7 have no common upper bound)")
print("  {1, 2, 3}   → NOT DIRECTED! (pairwise incomparable)")

## 5. Lazy Lists are dcpos

The domain of lazy lists is also a dcpo.

Chains of partial lists have limits:
```
⊔{⊥, 1:⊥, 1:2:⊥, 1:2:3:⊥, ...} = [1,2,3,...]
```

Every element in the chain contributes information to the limit.

In [ ]:
# The ones list approximation
ones = fixpoints_df[fixpoints_df['function_name'] == 'ones_list']

print("Approximating the infinite list of ones:")
print()
for _, row in ones.iterrows():
    if row['is_fixed_point']:
        print(f"  ⊔ = {row['current_approximation']} (the supremum)")
    else:
        print(f"  Step {row['iteration']}: {row['current_approximation']}")

## 6. Pointed dcpos

A **pointed dcpo** is a dcpo with a least element ⊥.

All the domains we use for denotational semantics are pointed dcpos. The bottom element represents:
- Non-termination
- Undefined values
- The starting point for fixed-point iteration

In [ ]:
# All our domains are pointed
print("Bottom elements ensure our dcpos are pointed:")
print()

bottoms = domains_df[domains_df['is_bottom'] == True]
for _, row in bottoms.iterrows():
    print(f"  {row['domain']}: ⊥ = {row['element']}")

## 7. Non-Example: The Integers

The integers ℤ with the usual order (... < -1 < 0 < 1 < ...) are **not** a dcpo for our purposes.

Why? The chain {0, 1, 2, 3, ...} has no supremum in ℤ.

For domain theory, we need the flat integers: {⊥, ..., -1, 0, 1, ...} where ⊥ is below everything and numbers are incomparable.

In [ ]:
# Contrast: usual order vs flat domain
print("Usual integer order (NOT a dcpo for our purposes):")
print("  ... < -1 < 0 < 1 < 2 < ...")
print("  The chain {0, 1, 2, ...} has no supremum!")
print()
print("Flat integer domain (IS a dcpo):")
print("        ... -1   0   1   2 ...  (incomparable)")
print("             \   |   /")
print("               ⊥              (bottom)")
print("  Every directed set has a supremum.")

## 8. Why Completeness Matters

The dcpo property is essential for:

1. **Fixed points exist**: Iteration from ⊥ converges
2. **Recursive definitions have meaning**: The limit of approximations exists
3. **Continuity makes sense**: We can talk about preserving limits

Without completeness, our chains would have nowhere to go.

In [ ]:
# Summary: dcpos for all our examples
print("Summary: Key domains and their dcpo status")
print()
print("| Domain | Has ⊥ | Directed-Complete | Used For |")
print("|--------|-------|-------------------|----------|")
print("| flat_nat | Yes | Yes | Natural numbers |")
print("| flat_bool | Yes | Yes | Booleans |")
print("| lazy_list | Yes | Yes | Lists, streams |")
print("| interval_domain | Yes | Yes | Real approximations |")
print("| function_domain | Yes | Yes | Higher-order functions |")
print("| reflexive_domain | Yes | Yes | Lambda calculus models |")

## Summary

In this tutorial, we learned:

1. **Directed sets** are sets where every pair has an upper bound
2. **Suprema** (⊔D) are least upper bounds of directed sets
3. **dcpos** are partial orders where every directed set has a supremum
4. **Flat domains** are dcpos with simple structure
5. **Lazy lists** form a dcpo with partial information
6. **Pointed dcpos** have a bottom element ⊥
7. **Completeness** ensures recursive definitions converge

In the next tutorial, we'll see which functions preserve this structure: **continuous functions**.

---

## Exercises

1. Is {{}, {a}, {a,b}, {a,b,c}} directed under the subset ordering? What is its supremum?

2. Give an example of a non-directed set in the lazy list domain.

3. Why is it important that our domains have a bottom element?

4. The power set P(ℕ) ordered by subset inclusion is a dcpo. What is the supremum of {{0}, {0,1}, {0,1,2}, ...}?

---

*Next: Tutorial 4 - Continuous Functions*